# Solveit Communication Patterns

This dialog explores the four bidirectional communication patterns between User Python (ipykernel), Solveit Python (FastHTML web app), and Browser JavaScript.

## The Four Patterns

1. **Python → JS blocking**: Python waits for JS to respond
2. **Python → JS non-blocking**: Python sends and continues
3. **JS → Python blocking**: JS waits for Python to respond
4. **JS → Python non-blocking**: JS sends and continues

## Architecture

```
User Python (ipykernel) ↔ Solveit Python (FastHTML:5001) ↔ Browser JS
```

## Key Files

- `dialoghelper/core.py`: `event_get()`, `fire_event()`, `add_html()`, `iife()`, `pop_data()`
- `selection.js`: `_post()`, `pushData()`
- `monedit.js`: `fetch()` to completion endpoints

## Related Dialogs

- Parent: `explorer` (main exploration session)
- Related: `architecture_deep_dive`, `extension_patterns`

## External References

- htmx documentation: https://htmx.org/
- Carson Gross on hypermedia: https://htmx.org/essays/

In [ ]:
enable_mermaid()

## Architecture Diagram

```mermaid
graph TB
    subgraph Browser
        JS[JavaScript]
    end
    
    subgraph Solveit[Solveit Python :5001]
        Routes[FastHTML Routes]
        WS[WebSocket]
    end
    
    subgraph User[User Python ipykernel]
        DH[dialoghelper]
        Code[User Code]
    end
    
    User -->|HTTP POST| Solveit
    Solveit -->|JSON Response| User
    Solveit -->|WebSocket Push| Browser
    Browser -->|htmx POST| Solveit
    Browser -->|fetch await| Solveit
    
    style Browser fill:#e1f5ff
    style Solveit fill:#fff4e1
    style User fill:#f0ffe1
```

## Communication Flow

```mermaid
sequenceDiagram
    participant UP as User Python
    participant SP as Solveit Python
    participant BR as Browser JS
    
    Note over UP,BR: Pattern 1: Python → JS Blocking
    UP->>SP: event_get('myEvent')
    SP->>BR: fire_event via WebSocket
    BR->>SP: pushData(uuid, result)
    SP->>UP: return result
    
    Note over UP,BR: Pattern 2: Python → JS Non-blocking
    UP->>SP: add_html(content)
    SP->>BR: WebSocket swap
    
    Note over UP,BR: Pattern 3: JS → Python Blocking
    BR->>SP: await fetch('/complete_')
    SP->>UP: call jedi
    UP->>SP: return completions
    SP->>BR: JSON response
    
    Note over UP,BR: Pattern 4: JS → Python Non-blocking
    BR->>SP: _post('save_dlg')
    SP->>BR: WebSocket update
```

## Terminology

- user python: python code running in the ipykernel process
- user state: transient state in ipykernel
- solveit python: python code running in a python interpreter, normal python process
- solveit state: the messages structure

# 1. Python -> JS blocking

## `event_get` and `pushData`

workflow pattern:
1. user python calls `event_get`

### Example - JS ipython magic

In [ ]:
# Register `js2Eval` handler to evaluate JavaScript code in the browser
HTML(to_xml(Script("""
console.log('Registering event listener: js2Eval');
// debugger;

window._js2EvalCtrl?.abort();
window._js2EvalCtrl = new AbortController();
document.body.addEventListener('js2Eval', e => {
    // Store `eval` in a variable so, when we invoke it, it's considered to
    // be an indirect eval and runs in the global rather than local scope.
    const globalEval = eval;
    
    try {
        const code = e.detail.code;
        const result = globalEval(code);
        pushData(e.detail.idx, {result: result});
    } catch (error) {
        pushData(e.detail.idx, { error: { name: error.name, message: error.message, stack: error.stack } });
    }
}, { signal: window._js2EvalCtrl.signal });
""")))

In [ ]:
def handle_js2_magic(code):
    evt = event_get('js2Eval', data={'code': code})
    if 'error' in evt:
        error = evt.error
        error_string = error.get('name', '<Unknown Error>')
        if 'message' in error: error_string += f": {error.message}"
        display(HTML(to_xml(Pre(error_string, style='background-color: #fcebeb;', cls='p-2'))))
    elif 'result' in evt:
        display(evt.result)

@register_line_magic
def js2(line): handle_js2_magic(line)

@register_cell_magic
def js2(line, cell): handle_js2_magic(cell)

In [ ]:
%%js2
1+2

In [ ]:
%%js2

function greet(name) {
    return `Hello, ${name}!`;
}

greet('world');

The `%%js2` magic demonstrates the full round-trip: Python blocks waiting for JS to `eval` code and return the result via `pushData`.

# 2. Python -> JS non-blocking

In [ ]:
@register_cell_magic
def fire(line, cell):
    evt_name = line.strip() or 'customEvent'
    fire_event(evt_name, {'payload': cell})

In [ ]:
%%fire testEvent
{"message": "Hello from fire_event", "timestamp": 123456}

Now register a listener in the browser to see it:

In [ ]:
%%js2
let received = null;
document.body.addEventListener('testEvent', e => {
    received = e.detail;
    console.log('Received event:', e.detail);
});
'Listener registered'

Then fire again and check:

In [ ]:
%%fire testEvent
{"data": "second message"}

In [ ]:
%js2 received

## Core Functions

**`add_html(content)`** - Inject HTML with `hx-swap-oob`
**`add_scr(scr)`** - Wrap JS in `<script>` and inject
**`iife(code)`** - Wrap JS in async IIFE
**`fire_event(evt, data)`** - Trigger htmx custom event

# 3. JS -> Python blocking

## Monaco Editor Completion Endpoints

From `monedit.js`, these are **JS → Python blocking** patterns where JS awaits Python responses:

- `/complete_` - Traditional autocomplete (Ctrl+Space)
- `/sig_help_` - Function signature hints
- `/ghost_` - AI ghost text completions
- `/supere_` - AI edit selected text
- `/superc_` - AI complete at cursor

All use `await fetch()` - JS blocks waiting for Python/LLM response.

# 4. JS -> Python non-blocking

```javascript
function _post(ep, payload = {}) {
  payload.dlg_name = $('#dlg_name').val();
  if (typeof payload.id_ !== 'string') payload.id_ = $pm.attr('id') || null;
  Object.assign(payload, { ids: selectedMsgIds() || null });
  htmx.ajax('POST', '/' + ep + '_', { values: payload, swap: 'none' });
}
```

**Purpose:** Standard fire-and-forget communication from browser to Solveit Python
- Auto-adds `dlg_name`, `id_` (current message), `ids` (all selected)
- Uses `swap: 'none'` - doesn't wait for response
- All keyboard shortcuts use this pattern

**Examples:** `_post('save_dlg')`, `_post('clear_out')`, `_post('toggle_pin')`

# Summary

## Pattern Comparison

| Pattern | Mechanism | Initiator | Example |
|---------|-----------|-----------|---------|
| Python → JS blocking | `event_get()` + `pushData()` | Python | `%%js2` magic |
| Python → JS non-blocking | `add_html()`, `fire_event()` | Python | Inject UI, trigger events |
| JS → Python blocking | `await fetch()` | JS | Completions, AI edits |
| JS → Python non-blocking | `_post()` via htmx | JS | Keyboard shortcuts |

## Key Insight

Solveit's hypermedia architecture makes all communication **declarative**:
- Python sends HTML/events, not API calls
- JS triggers endpoints, server decides response
- State updates flow through WebSocket
- Browser is a view, Solveit Python is the source of truth